In [107]:
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import keras
from keras.models import Model
from keras.layers import Input, Conv1D, MaxPooling1D, Dropout, Dense,Flatten, Conv2D, MaxPooling2D
from keras.optimizers import Adam
from keras import backend as K

from sklearn.model_selection import KFold


In [86]:
path = 'D:/Weina/vrsickness/data/processed/'
x_path = path + "Normalized_FinalTimeSquenceFeatures/"
y_path = path + "Ratings/"


In [87]:
# Load datafile name and path 
#x_files = []
filenames = [] # filename for the 2,400 data points
pID_set = set()
gmID_set = set()
for root, dirs, files in os.walk(x_path):
    for name in files:
        if name[-3:] == 'csv':
            path = os.path.join(root, name)
            fn = path.split('/')[-1]
            filenames.append(fn) # example: 'P02_A_10.csv'
            pID, gmID, _ = fn[:-4].split('_')
            pID_set.add(pID)
            gmID_set.add(gmID)
pID_list = list(pID_set)

In [109]:
class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, list_IDs, batch_size, x_path, y_path, filenames, n_timestep = 60, n_channels = 18,  shuffle=True):
        'Initialization'
        self.list_IDs = list_IDs
        self.filenames = filenames
        self.n_timestep = n_timestep
        self.n_channels = n_channels
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.target = pd.read_csv('D:/Weina/vrsickness/data/target_final.csv')


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        list_IDs_temp = [self.list_IDs[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(list_IDs_temp)

        return X, y


    def __data_generation(self, list_IDs_temp):
        'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, self.n_timestep, self.n_channels))
        y = np.empty((self.batch_size), dtype=float)

        # Generate data
        for i, ID in enumerate(list_IDs_temp):
            # Store sample
            X[i,] = np.array(pd.read_csv(x_path+filenames[ID], header = None))

            # Store score
            # y[i] = np.array(pd.read_csv(y_path+filenames[ID], header = None)).squeeze()
            pID, gmID, _ = filenames[ID][:-4].split('_')            
            y[i] = int(self.target.loc[(self.target['GameID']==gmID) & (self.target['ID']==pID) ]['Rating'])/10.0

        return X, y
    
    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)    

In [110]:
def coeff_determination(y_true, y_pred):
    SS_res =  K.sum(K.square( y_true-y_pred )) 
    SS_tot = K.sum(K.square( y_true - K.mean(y_true) ) ) 
    return ( 1 - SS_res/(SS_tot + K.epsilon()) )

def make_model(n1, n2, lr, dropout):
    x = Input(shape=(60,18))   # sequences of 60 vectors of 18-dimensional vectors
    model = Conv1D(n1, 3, strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'glorot_uniform')(x)
    #model = MaxPooling1D(2, strides=2, padding='same')(model)
    model = Conv1D(n2, 3, strides = 1, padding = 'same', activation = 'relu', kernel_initializer = 'glorot_uniform')(model)
    model = MaxPooling1D(2, strides=2, padding='same')(model)
    model = Flatten()(model)
    model = Dropout(dropout)(model)
    model = Dense(1, activation = 'relu', kernel_initializer = 'glorot_uniform')(model)
    
    model = Model(inputs = x, outputs = model)
    
    optimizer = Adam(lr=lr)
    model.compile(optimizer = optimizer , loss = 'mse', metrics = ['mse',coeff_determination])
    
    return model
    
#model = make_model()    

In [111]:
def get_train_test_generator(train_pID, test_pID, batch_size):
    """1. Generate the filename list from the pIDs;
    2. Return the data generator according to the filename list
    """
    train_index = []
    test_index = []
    for i in range(len(filenames)):
        pID, gmID, _ = filenames[i][:-4].split('_')
        if pID in test_pID:
            test_index.append(i)
        if pID in train_pID:
            train_index.append(i)

    train_generator = DataGenerator(train_index, batch_size, **params)
    test_generator = DataGenerator(test_index,batch_size,  **params)
    return train_generator, test_generator

In [144]:
# Parameters for building the data generator
params = {'x_path': x_path,
          'y_path': y_path,
          'filenames': filenames}

inner_epochs = 10 #80
outer_epochs = 60 #300
trials = 10 # random search times

outer_k = 8 #8  # 3 person for test, 21 for trainval 
inner_k = 7 #7  # 3 person for val, 18 for train

In [145]:

outer_kf = KFold(n_splits=outer_k, shuffle= True)
saved_data = []
metrics = []
for trainval_idx, test_idx in outer_kf.split(pID_list):
    # split outer k-fold
    trainval_IDs = np.array(pID_list)[trainval_idx]
    test_IDs = np.array(pID_list)[test_idx]

    # for each outer k-fold, use grid search to find the best model
    model_selection = {}
    for trial in range(trials):
        # 1. generate one hyperparameter settings, and save with a dict 
        n1 = 2**np.random.randint(5, 9) # 32-256
        n2 = 2**np.random.randint(5, 9) # 32-256
        lr = 10**(-3*np.random.rand()-2)  # [-2, -5]
        batch_size = 2**np.random.randint(4, 8) # 16, 32, 64, 128
        dropout = np.random.rand() * 0.4 + 0.4 # 0.4-0.8
        hyperpara_dict = {"n1": n1, "n2": n2, "lr": lr, "batch_size": batch_size, "dropout": dropout}
        print(hyperpara_dict)
        # 2. validate the hyperparameter settings with the inner CV
        # save the performance over the inner k-fold
        inner_kf = KFold(n_splits=inner_k, shuffle= True)
        for train_idx, val_idx in inner_kf.split(trainval_IDs): 
            train_IDs = np.array(trainval_IDs)[train_idx]
            val_IDs = np.array(trainval_IDs)[val_idx]
            train_gen, val_gen = get_train_test_generator(train_IDs, val_IDs, batch_size = batch_size)
            model = make_model(n1, n2, lr, dropout)
            history = model.fit_generator(generator=train_gen, validation_data=val_gen,
                    shuffle=True, verbose = 1, epochs = inner_epochs)
            R2 = history.history['val_coeff_determination']
            mse =history.history['val_mean_squared_error']
        # save the hyperpara dict and the performance
        model_selection[mse[-1]] = (R2[-1], hyperpara_dict)     
        
    # get the model with the best performance with mse
    min_mse = np.array(list(model_selection.keys())).min()
    best_para = model_selection[min_mse][1]
    # get data generator and train model
    train_gen, test_gen = get_train_test_generator(trainval_IDs, test_IDs, batch_size = best_para['batch_size'])
    model = make_model(best_para['n1'], best_para['n2'], best_para['lr'], best_para['dropout'])
    history = model.fit_generator(generator=train_gen, validation_data=test_gen,
        shuffle=True, verbose = 1, epochs = outer_epochs)
    R2 =history.history['val_coeff_determination']
    mse = history.history['val_mean_squared_error']
    saved_data.append(model_selection)
    metrics.append([mse[-1], R2[-1]])
    print("  ")
    print("outer fold best model mse :", min_mse, " with hyperpara: ", best_para)
    print("test mse: ", mse[-1])
    print('**************************')
    print(" ")
    
                                            
                                            
                                            



{'dropout': 0.44607983858637024, 'n1': 32, 'batch_size': 16, 'lr': 0.0003192704727214547, 'n2': 128}
Epoch 1/10
112/112 [==============================] - 16s 144ms/step - loss: 0.2024 - mean_squared_error: 0.2024 - coeff_determination: -1.9988 - val_loss: 0.0990 - val_mean_squared_error: 0.0990 - val_coeff_determination: -0.8675
Epoch 2/10
112/112 [==============================] - 9s 84ms/step - loss: 0.1450 - mean_squared_error: 0.1450 - coeff_determination: -1.1098 - val_loss: 0.0854 - val_mean_squared_error: 0.0854 - val_coeff_determination: -0.6376
Epoch 3/10
112/112 [==============================] - 9s 85ms/step - loss: 0.1158 - mean_squared_error: 0.1158 - coeff_determination: -0.7699 - val_loss: 0.0739 - val_mean_squared_error: 0.0739 - val_coeff_determination: -0.4668
Epoch 4/10
112/112 [==============================] - 9s 84ms/step - loss: 0.1037 - mean_squared_error: 0.1037 - coeff_determination: -0.5582 - val_loss: 0.0693 - val_mean_squared_error: 0.0693 - val_coeff_dete

112/112 [==============================] - 10s 85ms/step - loss: 0.0647 - mean_squared_error: 0.0647 - coeff_determination: -0.0621 - val_loss: 0.1252 - val_mean_squared_error: 0.1252 - val_coeff_determination: -0.4086
Epoch 7/10
112/112 [==============================] - 10s 85ms/step - loss: 0.0525 - mean_squared_error: 0.0525 - coeff_determination: 0.0925 - val_loss: 0.1214 - val_mean_squared_error: 0.1214 - val_coeff_determination: -0.3713
Epoch 8/10
112/112 [==============================] - 10s 85ms/step - loss: 0.0465 - mean_squared_error: 0.0465 - coeff_determination: 0.2081 - val_loss: 0.1194 - val_mean_squared_error: 0.1194 - val_coeff_determination: -0.3668
Epoch 9/10
112/112 [==============================] - 10s 85ms/step - loss: 0.0447 - mean_squared_error: 0.0447 - coeff_determination: 0.2632 - val_loss: 0.1190 - val_mean_squared_error: 0.1190 - val_coeff_determination: -0.3717
Epoch 10/10
112/112 [==============================] - 10s 85ms/step - loss: 0.0435 - mean_squ

112/112 [==============================] - 15s 136ms/step - loss: 0.2045 - mean_squared_error: 0.2045 - coeff_determination: -2.3583 - val_loss: 0.1926 - val_mean_squared_error: 0.1926 - val_coeff_determination: -0.9810
Epoch 2/10
112/112 [==============================] - 10s 85ms/step - loss: 0.1607 - mean_squared_error: 0.1607 - coeff_determination: -1.5733 - val_loss: 0.1860 - val_mean_squared_error: 0.1860 - val_coeff_determination: -0.9163
Epoch 3/10
112/112 [==============================] - 10s 85ms/step - loss: 0.1738 - mean_squared_error: 0.1738 - coeff_determination: -1.8237 - val_loss: 0.1863 - val_mean_squared_error: 0.1863 - val_coeff_determination: -0.9210
Epoch 4/10
112/112 [==============================] - 10s 86ms/step - loss: 0.1699 - mean_squared_error: 0.1699 - coeff_determination: -1.7037 - val_loss: 0.1832 - val_mean_squared_error: 0.1832 - val_coeff_determination: -0.8910
Epoch 5/10
112/112 [==============================] - 10s 86ms/step - loss: 0.2067 - mean_

Epoch 7/10
112/112 [==============================] - 10s 85ms/step - loss: 0.1630 - mean_squared_error: 0.1630 - coeff_determination: -1.4651 - val_loss: 0.1707 - val_mean_squared_error: 0.1707 - val_coeff_determination: -1.6382
Epoch 8/10
112/112 [==============================] - 10s 87ms/step - loss: 0.1704 - mean_squared_error: 0.1704 - coeff_determination: -1.7274 - val_loss: 0.1715 - val_mean_squared_error: 0.1715 - val_coeff_determination: -1.6484
Epoch 9/10
112/112 [==============================] - 10s 86ms/step - loss: 0.1538 - mean_squared_error: 0.1538 - coeff_determination: -1.3410 - val_loss: 0.1636 - val_mean_squared_error: 0.1636 - val_coeff_determination: -1.5419
Epoch 10/10
112/112 [==============================] - 10s 88ms/step - loss: 0.1691 - mean_squared_error: 0.1691 - coeff_determination: -1.6983 - val_loss: 0.1632 - val_mean_squared_error: 0.1632 - val_coeff_determination: -1.5388
Epoch 1/10
112/112 [==============================] - 16s 139ms/step - loss: 0.

14/14 [==============================] - 9s 609ms/step - loss: 0.1632 - mean_squared_error: 0.1632 - coeff_determination: -1.3533 - val_loss: 0.2612 - val_mean_squared_error: 0.2612 - val_coeff_determination: -2.8345
Epoch 3/10
14/14 [==============================] - 9s 615ms/step - loss: 0.1636 - mean_squared_error: 0.1636 - coeff_determination: -1.3528 - val_loss: 0.2612 - val_mean_squared_error: 0.2612 - val_coeff_determination: -2.8345
Epoch 4/10
14/14 [==============================] - 9s 613ms/step - loss: 0.1636 - mean_squared_error: 0.1636 - coeff_determination: -1.3636 - val_loss: 0.2612 - val_mean_squared_error: 0.2612 - val_coeff_determination: -2.8345
Epoch 5/10
14/14 [==============================] - 8s 598ms/step - loss: 0.1629 - mean_squared_error: 0.1629 - coeff_determination: -1.3439 - val_loss: 0.2612 - val_mean_squared_error: 0.2612 - val_coeff_determination: -2.8345
Epoch 6/10
14/14 [==============================] - 8s 605ms/step - loss: 0.1630 - mean_squared_err

...

28/28 [==============================] - 70s 3s/step - loss: 0.1560 - mean_squared_error: 0.1560 - coeff_determination: -1.4554 - val_loss: 0.0637 - val_mean_squared_error: 0.0637 - val_coeff_determination: -1.0533
Epoch 4/10
28/28 [==============================] - 52s 2s/step - loss: 0.1323 - mean_squared_error: 0.1323 - coeff_determination: -1.0516 - val_loss: 0.0690 - val_mean_squared_error: 0.0690 - val_coeff_determination: -1.1750
Epoch 5/10
28/28 [==============================] - 62s 2s/step - loss: 0.1241 - mean_squared_error: 0.1241 - coeff_determination: -0.9176 - val_loss: 0.0806 - val_mean_squared_error: 0.0806 - val_coeff_determination: -1.5540
Epoch 6/10
28/28 [==============================] - 62s 2s/step - loss: 0.0942 - mean_squared_error: 0.0942 - coeff_determination: -0.4477 - val_loss: 0.0678 - val_mean_squared_error: 0.0678 - val_coeff_determination: -1.1451
Epoch 7/10
28/28 [==============================] - 68s 2s/step - loss: 0.0760 - mean_squared_error: 0.0760

Epoch 9/10
14/14 [==============================] - 55s 4s/step - loss: 0.1272 - mean_squared_error: 0.1272 - coeff_determination: -0.9237 - val_loss: 0.0675 - val_mean_squared_error: 0.0675 - val_coeff_determination: -0.7544
Epoch 10/10
14/14 [==============================] - 63s 5s/step - loss: 0.1210 - mean_squared_error: 0.1210 - coeff_determination: -0.8375 - val_loss: 0.0567 - val_mean_squared_error: 0.0567 - val_coeff_determination: -0.4785
Epoch 1/10
14/14 [==============================] - 224s 16s/step - loss: 0.1779 - mean_squared_error: 0.1779 - coeff_determination: -1.7850 - val_loss: 0.1709 - val_mean_squared_error: 0.1709 - val_coeff_determination: -1.9743
Epoch 2/10
14/14 [==============================] - 54s 4s/step - loss: 0.1431 - mean_squared_error: 0.1431 - coeff_determination: -1.2596 - val_loss: 0.1521 - val_mean_squared_error: 0.1521 - val_coeff_determination: -1.6524
Epoch 3/10
14/14 [==============================] - 60s 4s/step - loss: 0.1325 - mean_squared

14/14 [==============================] - 61s 4s/step - loss: 0.1428 - mean_squared_error: 0.1428 - coeff_determination: -1.2418 - val_loss: 0.1092 - val_mean_squared_error: 0.1092 - val_coeff_determination: -1.0935
Epoch 6/10
14/14 [==============================] - 55s 4s/step - loss: 0.1420 - mean_squared_error: 0.1420 - coeff_determination: -1.2308 - val_loss: 0.1019 - val_mean_squared_error: 0.1019 - val_coeff_determination: -0.9099
Epoch 7/10
14/14 [==============================] - 72s 5s/step - loss: 0.1340 - mean_squared_error: 0.1340 - coeff_determination: -1.1150 - val_loss: 0.1130 - val_mean_squared_error: 0.1130 - val_coeff_determination: -1.0034
Epoch 8/10
14/14 [==============================] - 64s 5s/step - loss: 0.1262 - mean_squared_error: 0.1262 - coeff_determination: -0.9878 - val_loss: 0.1024 - val_mean_squared_error: 0.1024 - val_coeff_determination: -0.8594
Epoch 9/10
14/14 [==============================] - 62s 4s/step - loss: 0.1254 - mean_squared_error: 0.1254

65/65 [==============================] - 83s 1s/step - loss: 0.0622 - mean_squared_error: 0.0622 - coeff_determination: -0.0269 - val_loss: 0.1198 - val_mean_squared_error: 0.1198 - val_coeff_determination: -0.4812
Epoch 12/60
65/65 [==============================] - 75s 1s/step - loss: 0.0579 - mean_squared_error: 0.0579 - coeff_determination: 0.0352 - val_loss: 0.1209 - val_mean_squared_error: 0.1209 - val_coeff_determination: -0.5337
Epoch 13/60
65/65 [==============================] - 73s 1s/step - loss: 0.0582 - mean_squared_error: 0.0582 - coeff_determination: 0.0187 - val_loss: 0.1175 - val_mean_squared_error: 0.1175 - val_coeff_determination: -0.4958
Epoch 14/60
65/65 [==============================] - 74s 1s/step - loss: 0.0526 - mean_squared_error: 0.0526 - coeff_determination: 0.1217 - val_loss: 0.1105 - val_mean_squared_error: 0.1105 - val_coeff_determination: -0.3461
Epoch 15/60
65/65 [==============================] - 70s 1s/step - loss: 0.0513 - mean_squared_error: 0.051

65/65 [==============================] - 142s 2s/step - loss: 0.0261 - mean_squared_error: 0.0261 - coeff_determination: 0.5702 - val_loss: 0.0865 - val_mean_squared_error: 0.0865 - val_coeff_determination: -0.0514
Epoch 48/60
65/65 [==============================] - 139s 2s/step - loss: 0.0253 - mean_squared_error: 0.0253 - coeff_determination: 0.5763 - val_loss: 0.0780 - val_mean_squared_error: 0.0780 - val_coeff_determination: 0.0514
Epoch 49/60
65/65 [==============================] - 143s 2s/step - loss: 0.0259 - mean_squared_error: 0.0259 - coeff_determination: 0.5665 - val_loss: 0.0941 - val_mean_squared_error: 0.0941 - val_coeff_determination: -0.2311
Epoch 50/60
65/65 [==============================] - 152s 2s/step - loss: 0.0253 - mean_squared_error: 0.0253 - coeff_determination: 0.5769 - val_loss: 0.0935 - val_mean_squared_error: 0.0935 - val_coeff_determination: -0.2268
Epoch 51/60
65/65 [==============================] - 150s 2s/step - loss: 0.0243 - mean_squared_error: 0.

In [146]:
# get the final generalization mse and R2
metrics = np.array(metrics)
mse_mean = metrics[:,0].mean()
R2_mean =  metrics[:,1].mean()

In [147]:
metrics

array([[ 0.04138295, -0.0972534 ],
       [ 0.04531049,  0.20230122],
       [ 0.05997196,  0.09764172],
       [ 0.07395983, -0.64345044],
       [ 0.05039667,  0.20904656],
       [ 0.06601982, -0.60370863],
       [ 0.06192676,  0.15361183],
       [ 0.09783706, -0.18728214]])

In [149]:
print("mse_mean: ", mse_mean, "R2_mean: ", R2_mean)

mse_mean:  0.0621006931242947 R2_mean:  -0.10863665863871574


In [152]:
np.savetxt("metrics_result.csv", metrics, delimiter=",")


In [153]:
saved_data

[{0.05859675206657913: (-0.008237729469935099,
   {'batch_size': 16,
    'dropout': 0.44607983858637024,
    'lr': 0.0003192704727214547,
    'n1': 32,
    'n2': 128}),
  0.07005224004387856: (-0.9105560647116767,
   {'batch_size': 16,
    'dropout': 0.6333860760649368,
    'lr': 2.403039635855008e-05,
    'n1': 128,
    'n2': 128}),
  0.07535211721228229: (-0.09891390469339159,
   {'batch_size': 16,
    'dropout': 0.5974323471564951,
    'lr': 0.0006164385451429984,
    'n1': 128,
    'n2': 64}),
  0.1374364255203141: (-0.9365139537387424,
   {'batch_size': 32,
    'dropout': 0.6024544322829183,
    'lr': 0.0007532260727299265,
    'n1': 32,
    'n2': 256}),
  0.14168259667025673: (-2.2669818136427136,
   {'batch_size': 32,
    'dropout': 0.47204345101658896,
    'lr': 1.154453701829593e-05,
    'n1': 128,
    'n2': 256}),
  0.15036657949288687: (-2.3519217703077526,
   {'batch_size': 32,
    'dropout': 0.7466728074115725,
    'lr': 0.0009214898283075703,
    'n1': 64,
    'n2': 64}),

In [156]:
import csv

with open('1D_cnn_saved_data.csv', 'w') as myfile:
    wr = csv.writer(myfile, lineterminator='\n')
    wr.writerow(saved_data)